In [1]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import f1_score

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score

import xgboost
from keras.wrappers.scikit_learn import KerasClassifier

import tensorflow as tf
from keras import backend as K

import numpy as np 
import pandas as pd

Using TensorFlow backend.


In [2]:
#import train dataset
tripDataTrain = pd.read_csv('train.csv')


#import test dataset
tripDataTest = pd.read_csv('test.csv')


#print null values count in train and test datasets
print(tripDataTrain.isnull().sum())

print(tripDataTest.isnull().sum())


tripDataTrain.shape
tripDataTest.shape


def remove_noise(col_name):
    n=3
    td = col_name
    median = tripDataTrain[td].median()
    std = tripDataTrain[td].std()
    outliers = (tripDataTrain[td] - median).abs() > std*n

    # Set outliers to NAN
    #tripDataTrain[outliers] = np.nan
    tripDataTrain.loc[(tripDataTrain[td] - median).abs() > std*n, td] = np.nan

    # Fill NAN with median
    tripDataTrain[td].fillna(median, inplace=True)



#replace null values with mean value
tripDataTrain.fillna(tripDataTrain.mean(), inplace=True)
tripDataTest.fillna(tripDataTest.mean(), inplace=True)


#Did not used because accuracy was reduced 
# remove_noise("fare")
# remove_noise('additional_fare')
# remove_noise('duration')
# remove_noise("meter_waiting")
# remove_noise('meter_waiting_fare')
# remove_noise('meter_waiting_till_pickup')


tripDataTrain['label']=tripDataTrain['label'].map({"correct":1,"incorrect":0})
tripDataTrain.shape


tripDataTrain['drop_time'] = pd.to_datetime(tripDataTrain['drop_time'], format = '%m/%d/%Y %H:%M', errors = 'coerce')
tripDataTrain['pickup_time'] = pd.to_datetime(tripDataTrain['pickup_time'],format = '%m/%d/%Y %H:%M',errors = 'coerce')

tripDataTest['drop_time'] = pd.to_datetime(tripDataTrain['drop_time'], format = '%m/%d/%Y %H:%M', errors = 'coerce')
tripDataTest['pickup_time'] = pd.to_datetime(tripDataTrain['pickup_time'],format = '%m/%d/%Y %H:%M',errors = 'coerce')



tripDataTrain['pickup_time_year'] = tripDataTrain['pickup_time'].dt.year
tripDataTrain['pickup_time_month'] = tripDataTrain['pickup_time'].dt.month
tripDataTrain['pickup_time_day'] = tripDataTrain['pickup_time'].dt.day
tripDataTrain['pickup_time_hour'] = tripDataTrain['pickup_time'].dt.hour
tripDataTrain['pickup_time_minute'] = tripDataTrain['pickup_time'].dt.minute

tripDataTest['pickup_time_year'] = tripDataTest['pickup_time'].dt.year
tripDataTest['pickup_time_month'] = tripDataTest['pickup_time'].dt.month
tripDataTest['pickup_time_day'] = tripDataTest['pickup_time'].dt.day
tripDataTest['pickup_time_hour'] = tripDataTest['pickup_time'].dt.hour
tripDataTest['pickup_time_minute'] = tripDataTest['pickup_time'].dt.minute


tripDataTrain['drop_time_year'] = tripDataTrain['drop_time'].dt.year
tripDataTrain['drop_time_month'] = tripDataTrain['drop_time'].dt.month
tripDataTrain['drop_time_day'] = tripDataTrain['drop_time'].dt.day
tripDataTrain['drop_time_hour'] = tripDataTrain['drop_time'].dt.hour
tripDataTrain['drop_time_minute'] = tripDataTrain['drop_time'].dt.minute

tripDataTest['drop_time_year'] = tripDataTest['drop_time'].dt.year
tripDataTest['drop_time_month'] = tripDataTest['drop_time'].dt.month
tripDataTest['drop_time_day'] = tripDataTest['drop_time'].dt.day
tripDataTest['drop_time_hour'] = tripDataTest['drop_time'].dt.hour
tripDataTest['drop_time_minute'] = tripDataTest['drop_time'].dt.minute



tripDataTrain['fare_for_duration'] = (tripDataTrain['fare'] - tripDataTrain['meter_waiting_fare'])/tripDataTrain['duration']
tripDataTrain['meter_waiting_to_duration'] = tripDataTrain['meter_waiting']/tripDataTrain['duration']
tripDataTrain['additional_fare_to_duration'] = tripDataTrain['additional_fare']/tripDataTrain['duration']
tripDataTrain['mtr_wating_fare_to_waiting_duration'] = tripDataTrain['meter_waiting_fare']/(tripDataTrain['meter_waiting']+tripDataTrain['meter_waiting_till_pickup'])

tripDataTrain['additional_fare_to_full_time'] = tripDataTrain['additional_fare']/(tripDataTrain['meter_waiting']+tripDataTrain['meter_waiting_till_pickup']+tripDataTrain['duration'])


tripDataTest['fare_for_duration'] = (tripDataTest['fare'] - tripDataTest['meter_waiting_fare'])/tripDataTest['duration']
tripDataTest['meter_waiting_to_duration'] = tripDataTest['meter_waiting']/tripDataTest['duration']
tripDataTest['additional_fare_to_duration'] = tripDataTest['additional_fare']/tripDataTest['duration']
tripDataTest['mtr_wating_fare_to_waiting_duration'] = tripDataTest['meter_waiting_fare']/(tripDataTest['meter_waiting']+tripDataTest['meter_waiting_till_pickup'])
tripDataTest['additional_fare_to_full_time'] = tripDataTest['additional_fare']/(tripDataTest['meter_waiting']+tripDataTest['meter_waiting_till_pickup']+tripDataTest['duration'])



tripDataTrain['fare_for_duration'] = tripDataTrain['fare_for_duration'].astype(np.float64)
tripDataTrain['meter_waiting_to_duration'] = tripDataTrain['meter_waiting_to_duration'].astype(np.float64)
tripDataTrain['additional_fare_to_duration'] = tripDataTrain['additional_fare_to_duration'].astype(np.float64)
tripDataTrain['mtr_wating_fare_to_waiting_duration'] = tripDataTrain['mtr_wating_fare_to_waiting_duration'].astype(np.float64)
tripDataTrain['additional_fare_to_full_time'] = tripDataTrain['additional_fare_to_full_time'].astype(np.float64)

tripDataTest['fare_for_duration'] = tripDataTest['fare_for_duration'].astype(np.float64)
tripDataTest['meter_waiting_to_duration'] = tripDataTest['meter_waiting_to_duration'].astype(np.float64)
tripDataTest['additional_fare_to_duration'] = tripDataTest['additional_fare_to_duration'].astype(np.float64)
tripDataTest['mtr_wating_fare_to_waiting_duration'] = tripDataTest['mtr_wating_fare_to_waiting_duration'].astype(np.float64)
tripDataTest['additional_fare_to_full_time'] = tripDataTest['additional_fare_to_full_time'].astype(np.float64)


for index, row in tripDataTrain.iterrows():
    lat1, lng1, lat2, lng2 = map(np.radians, (row['drop_lat'], row['drop_lon'], row['pick_lat'], row['pick_lon']))
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * 6371 * np.arcsin(np.sqrt(d))
    tripDataTrain.at[index,'direct_distance'] = 2 * 6371 * np.arcsin(np.sqrt(d))
    
    
for index, row in tripDataTest.iterrows():
    lat1, lng1, lat2, lng2 = map(np.radians, (row['drop_lat'], row['drop_lon'], row['pick_lat'], row['pick_lon']))
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * 6371 * np.arcsin(np.sqrt(d))
    tripDataTest.at[index,'direct_distance'] = 2 * 6371 * np.arcsin(np.sqrt(d))



feature_list = ['additional_fare',  'additional_fare_to_full_time','mtr_wating_fare_to_waiting_duration', 'meter_waiting_to_duration', 'fare_for_duration', 'additional_fare_to_duration','drop_lat', 'drop_lon', 'pick_lat', 'pick_lon', 'duration','direct_distance', 'meter_waiting', 'meter_waiting_fare', 'meter_waiting_till_pickup', 'fare']


train_X_ = tripDataTrain[feature_list]
test_X = tripDataTest[feature_list]

train_X_.replace([np.inf, -np.inf], np.nan)
test_X.replace([np.inf, -np.inf], np.nan)
train_X_.fillna(train_X_.mean(), inplace=True)
test_X.fillna(test_X.mean(), inplace=True)

train_y_ = tripDataTrain.label


#correlation diagram
import matplotlib.pyplot as plt

x_cols = [col for col in tripDataTrain.columns if col not in ['label'] if tripDataTrain[col].dtype=='float64']

labels = []
values = []
for col in x_cols:
    labels.append(col)
    values.append(np.corrcoef(tripDataTrain[col].values, tripDataTrain.label.values)[0,1])
corr_df = pd.DataFrame({'col_labels':labels, 'corr_values':values})
corr_df = corr_df.sort_values(by='corr_values')

ind = np.arange(len(labels))
width = 0.9
fig, ax = plt.subplots(figsize=(10,10))
rects = ax.barh(ind, np.array(corr_df.corr_values.values), color='y')
ax.set_yticks(ind)
ax.set_yticklabels(corr_df.col_labels.values, rotation='horizontal')
ax.set_xlabel("Correlation coefficient")
ax.set_title("Correlation coefficient of the variables")
plt.show()



#split train and test dataset
X_train, X_test, y_train, y_test = train_test_split(train_X_, train_y_, test_size = 0.2, train_size = 0.8, random_state = 0)



def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)

    score=cross_val_score(model, test_features, test_labels, cv=10)
    accuracy = accuracy_score(test_labels, predictions.round())
    print('Accuracy:%f'%accuracy)



from sklearn.ensemble import RandomForestClassifier

#Random forest model
clf=RandomForestClassifier(n_estimators=100)
clf.fit(X_train,y_train)


evaluate(clf, X_test, y_test)

pred_classes = clf.predict(X_test)
predi = []
for x in range(len(pred_classes)):
    predi.append(bool(pred_classes[x]))
    
pred_classes = np.array(predi)

f1 = f1_score(y_test, pred_classes)
print('F1 score: %f' % f1)



#xg model 
xg_model = xgboost.XGBClassifier()
score=cross_val_score(xg_model, X_train, y_train, cv=10)
xg_model.fit(X_train,y_train)


evaluate(xg_model, X_test, y_test)

pred_classes = xg_model.predict(X_test)
predi = []
for x in range(len(pred_classes)):
    predi.append(bool(pred_classes[x]))
    
pred_classes = np.array(predi)

f1 = f1_score(y_test, pred_classes)
print('F1 score: %f' % f1)


#predict the values
prediction = xg_model.predict(test_X)


# create submission output file
pred_array = []
for x in range(len(prediction)):
    pred_array.append(bool(prediction[x]))
    
prediction = np.array(pred_array)

output = pd.DataFrame({'tripid': tripDataTest.tripid,
                       'prediction': prediction})

output_file = "submission_final.csv"
output.to_csv(output_file, index = False)


tripid                         0
additional_fare              202
duration                     202
meter_waiting                202
meter_waiting_fare           202
meter_waiting_till_pickup    202
pickup_time                    0
drop_time                      0
pick_lat                       0
pick_lon                       0
drop_lat                       0
drop_lon                       0
fare                         137
label                          0
dtype: int64
tripid                       0
additional_fare              0
duration                     0
meter_waiting                0
meter_waiting_fare           0
meter_waiting_till_pickup    0
pickup_time                  0
drop_time                    0
pick_lat                     0
pick_lon                     0
drop_lat                     0
drop_lon                     0
fare                         0
dtype: int64


C:\Users\Rasika_105127\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\generic.py:5434: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._update_inplace(new_data)


<Figure size 1000x1000 with 1 Axes>

Accuracy:0.956927
F1 score: 0.976530
Accuracy:0.958964
F1 score: 0.977580
